In [1]:
# pip install pandas
# pip install numpy
# !pip install transformers -U
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# pip install accelerate -U
# pip install transformers[torch]
# pip install accelerate>=0.21.0

In [31]:
import pandas as pd
data = pd.read_csv(r"D:\Final Model Training\Data Set\Final_data_set_shuffle_for_model.csv")
data.head()

,Headline,Label
0,bBelgian cardinals prevent priests to be in a ...,0
1,Majority of Russians and Americans View Each O...,0
2,Mr. Koistinen joins from Nokia Siemens Network...,0
3,Blatter to stand again for footballs top job,0
4,Brazils Army Moves To Protect Indigenous Aw Tr...,1


In [32]:
data["Label"].value_counts()
# data.columns

Label
1    62874
0    61492
Name: count, dtype: int64

In [33]:
data = data[['Headline','Label']]
data = data[0:10]
data.Label.value_counts()

Label
0    7
1    3
Name: count, dtype: int64

In [34]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import torch
from transformers import TrainingArguments,Trainer
from transformers import BertTokenizer,BertForSequenceClassification

In [35]:
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-uncased',num_labels=2)
model = model.to('cpu') # if CPU is available
# model = model.to('cuda') # if GPU is available

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# padding  = Ture Make all the list at same list add extra 0 to the list
# truncation = Ture which wil be truncate any text which has more than 512 words. Which maximum context size is the maximun amount of words a model can accept  
# return_tensors=True Convert normal python list to the tensorflow tensor
sample_data = ["Hello World","My name is Arsalan Bakhtiar"]
tokenizer(sample_data ,padding = True , truncation= True , max_length= 512)

{'input_ids': [[101, 7592, 2088, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2026, 2171, 2003, 29393, 7911, 2078, 8670, 10023, 10711, 2099, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [37]:
X = list(data['Headline'])
y = list(data['Label'])
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length= 512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [38]:
len(X_train),len(X_val)

(8, 2)

In [39]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [40]:
print(X_train_tokenized['input_ids'][0])

[101, 2720, 1012, 12849, 2923, 21820, 9794, 2013, 22098, 22108, 6125, 2073, 2002, 2038, 2218, 2536, 3026, 4341, 2968, 1998, 2449, 2458, 4460, 2144, 2722, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [41]:
sample_data = ["Hello World","My name is Arsalan Bakhtiar"]
tokenizer(sample_data ,padding = True , truncation= True , max_length= 512)

{'input_ids': [[101, 7592, 2088, 102, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2026, 2171, 2003, 29393, 7911, 2078, 8670, 10023, 10711, 2099, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [42]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [43]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [44]:
# train_dataset[5]

In [45]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [47]:
# Define Trainer
# args = TrainingArguments(
#     output_dir="output",
#     num_train_epochs = 4,
#     per_device_train_batch_size=8

# )
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)
    

In [48]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=4, training_loss=0.586188793182373, metrics={'train_runtime': 27.5357, 'train_samples_per_second': 1.162, 'train_steps_per_second': 0.145, 'total_flos': 871555370880.0, 'train_loss': 0.586188793182373, 'epoch': 4.0})

In [49]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


C:\Users\Arsalan Bakhtiar AB\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 0.6601483821868896,
 'eval_accuracy': 0.5,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_runtime': 0.6092,
 'eval_samples_per_second': 3.283,
 'eval_steps_per_second': 1.641,
 'epoch': 4.0}

In [ ]:
np.set_printoptions(suppress=True)

In [50]:
text = "In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cpu')
outputs = model(**inputs)
# print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
# print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.61435443, 0.3856456 ]], dtype=float32)

In [51]:
trainer.save_model("google-bert/bert-base-uncased-512-4 epoch")

In [52]:
import torch
from transformers import BertTokenizer,BertForSequenceClassification
model_2 = BertForSequenceClassification.from_pretrained("Fine_tuned_model")
model_2.to('cpu')

OSError: Fine_tuned_model is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import torch
# text = "That was good point"
text = "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m "
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cpu')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions
x = predictions[0, 0]
y = predictions[0, 1]
print(predictions)
print("X :",x)
print("Y :",y)

In [ ]:
# first one is 0 and second 1 is 1

In [11]:
print(predictions)
print("X :",x)
print("Y :",y)

[[0.01005797 0.98994195]]
X : 0.010057974
Y : 0.98994195


In [ ]:
from transformers import TFAutoModel,
                         AutoTokenizer 

In [ ]:
# Download the model form the internet
model = TFAutoModel.from_pretrained("bert-base-uncased")

In [ ]:
# Tokenizer
tokenizer = TFAutoModel.from_pretrained("bert-base-uncased")







In [ ]:
# padding  = Ture Make all the list at same list add extra 0 to the list
# truncation = Ture which wil be truncate any text which has more than 512 words. Which maximum context size is the maximun amount of words a model can accept  
# return_tensors=True Convert normal python list to the tensorflow tensor
inputs = tokenizer("Hello World",'Hi how are',padding = True, truncation = True, return_tensors = True)

In [ ]:
# call bert to train the model
output = model(inputs)